In [ ]:
from data.function import Stock,Categories
import plotly.graph_objects as go
import pandas as pd
import datetime
import numpy as np
from scipy import stats

In [ ]:



def convert_interval(data,period):
    if period not in ["MS","QS"]:
         raise Exception("Interval offset is not support. The support interval offset are 'MS' and 'QS'")
    # Use the resample function to get the open price on the first day of the month
    new_open = data['open'].resample(period).first()

    # Use the resample function to get the close price on the last day of the month
    new_close = data['close'].resample(period).last()

    # Use the resample function to get the maximum and minimum values for each month
    new_high = data['high'].resample(period).max()
    new_low = data['low'].resample(period).min()

    new_ma50 = data['MA50'].resample(period).mean()
    new_ma200 = data['MA200'].resample(period).mean()
    # Use the resample function to get the total volume for each month
    new_volume = data['volume'].resample(period).sum()
    # Create a new DataFrame with the monthly stock prices
    new_prices = pd.DataFrame({'open': new_open,
                                    'high': new_high,
                                    'low': new_low,
                                    'close': new_close,                                
                                    'volume': new_volume,
                                    'MA50': new_ma50,
                                    'MA200': new_ma200,
                                    })
    new_prices = new_prices.reset_index()
    return new_prices

def get_close_date(data,frequency):
    if frequency not in ["1d","1h"]:
         raise Exception("This frequency is not support. The support frequency are '1d' and '1h'")
    # grab first and last observations from df.date and make a continuous date range from that
    dt_all = pd.date_range(start=data['x'].iloc[0],end=data['x'].iloc[-1], freq = frequency)

    # check which dates from your source that also accur in the continuous date range
    dt_obs = [d.strftime("%Y-%m-%d %H:%M:%S") for d in data['x']]

    # isolate missing timestamps
    dt_breaks = [d for d in dt_all.strftime("%Y-%m-%d %H:%M:%S").tolist() if not d in dt_obs]
    dt_breaks = pd.to_datetime(dt_breaks)
    return dt_breaks

def candle_plot(data,close_day,name):
    #Candlestick Chart
    candle = go.Candlestick(**data.drop(columns=['volume','MA50','MA200']),yaxis = 'y2',name=name)

    #Moving Average 50
    MA50 = go.Scatter(x=data.x, y=data.MA50, line=dict(color='cyan', width=1.5),yaxis = 'y2',name='MA50')

    #Moving Average 200
    MA200 = go.Scatter(x=data.x, y=data.MA200, line=dict(color='#E377C2', width=1.5),yaxis = 'y2',name='MA200')

    #Volume
    volume = go.Bar(x=data.x,y=data.volume,marker={ "color": "gray"},yaxis = 'y',name='Volume')

    trace = [candle,MA50,MA200,volume]

    layout = {
    "xaxis": {"rangeselector": {
        "x": 0, 
        "y": 0.9, 
        "font": {"size": 13}, 
        "visible": True, 
        "bgcolor": "rgba(150, 200, 250, 0.4)", 
        }}, 
    "yaxis": {
        "domain": [0, 0.2], 
        "showticklabels": False
    }, 
    "legend": {
        "x": 0.3, 
        "y": 0.9, 
        "yanchor": "bottom", 
        "orientation": "h"
    }, 
    "margin": {
        "b": 40, 
        "l": 40, 
        "r": 40, 
        "t": 40
    }, 
    "yaxis2": {"domain": [0.2, 0.8]}, 
    "plot_bgcolor": "rgb(250, 250, 250)",
    "title":name+" Candlestick Chart"
    }

    fig = go.Figure(data=trace, layout=layout)
    if len(data['x']) <= 1:# If data have 1 data or less than
        pass
    elif str(data['x'][1]- data['x'][0]) == '0 days 01:00:00':# If interval is 1 hour
        fig.update_xaxes(rangebreaks=[dict(dvalue = 60*60*1000, values=close_day)])
    elif data['x'][1] - data['x'][0] <= pd.Timedelta(days=7):# If interval is less than 7 day
        fig.update_xaxes(rangebreaks=[dict(values=close_day)])
    fig.update_layout(hovermode='x')
    return fig
    # fig.show()

def plot(stock_symbol):
    
    plot_interval = ['Hourly','Daily','Monthly','Quarterly']

    # stock_symbol = symbol
    PTT = Stock(stock_symbol)
    pic = {}
    stock_price = []
    for round in plot_interval:
        pull_interval,interval_symbol = '',''
        if round == 'Hourly':
            pull_interval = '1h'
            stock_price = PTT.get_all_stock_price(interval=pull_interval)
        elif round in ['Daily','Monthly','Quarterly']:
            pull_interval = '1d'
            if round == 'Monthly':
                interval_symbol = 'MS'
                stock_price = PTT.get_all_stock_price(interval=pull_interval)
            elif round == 'Quarterly':
                interval_symbol = 'QS'
            else:#Daily
                last_date = PTT.latest_update_time(interval=pull_interval)
                start = datetime.datetime.strptime(last_date, '%Y-%m-%d %H:%M:%S')# convert string to datetime obj
                start = (start - datetime.timedelta(days=365)).strftime("%Y-%m-%d %H:%M:%S")
                stock_price = PTT.get_all_stock_price(interval=pull_interval,start=start)


        stock_price_df = pd.DataFrame(stock_price)
        stock_price_df = stock_price_df.rename(columns={0: 'x', 1: 'open', 2: 'high', 3: 'low', 4: 'close',5:'volume'})
        stock_price_df['x'] = pd.to_datetime(stock_price_df['x'])
        stock_price_df['MA50'] = stock_price_df.close.rolling(50).mean()
        stock_price_df['MA200'] = stock_price_df.close.rolling(200).mean() 
        stock_price_df.set_index('x', inplace=True)# Set the Date column as the index of the DataFrame

        if round in ['Hourly','Daily']:
            stock_price_df.reset_index(inplace=True)
        else:
            stock_price_df = convert_interval(stock_price_df,interval_symbol)# Convert daily data to monyhly or quarterly

        close_day = get_close_date(stock_price_df,pull_interval)# Find Close date
        pic[round]=candle_plot(stock_price_df,close_day,stock_symbol)
    return pic

fig = plot("PTT")
for p in fig.keys():
    fig[p].show()


In [ ]:
def finance_scater_plot_1_pic(name,fs,financials_data):
    x = list(range(len(fs[financials_data])))
    y = list(fs[financials_data])

    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

    x_line = np.linspace(min(x), max(x), len(fs[financials_data]))
    y_line = slope * x_line + intercept
    # Create the line object
    percent = (slope/abs(intercept))*100

    data_line = go.Scatter(x=fs["quarter"],y=fs[financials_data],mode='lines+markers', name=financials_data)
    regression = go.Scatter(x=fs["quarter"], y=y_line, mode='lines', name='Regression Line')

    # Create a subplot with 2 vertical axes
    fig = go.Figure(data = [data_line,regression])

    fig.update_layout(
        title=name+" "+financials_data,
        hovermode='x'
    )
    fig.update_xaxes(tickangle=60)
    return (fig,slope,percent,intercept)

def finance_bar_plot_1_pic(name,fs,financials_data):
    layout = {
        "yaxis": {
            "domain": [0, 0.8], 
            "showticklabels": True
        }, 
        "legend": {
            "x": 0.4, 
            "y": 0.9, 
            "yanchor": "bottom", 
            "orientation": "h"
        }, 
        "margin": {
            "b": 40, 
            "l": 40, 
            "r": 40, 
            "t": 40
        }, 
        "plot_bgcolor": "rgb(250, 250, 250)"
        }

    roa = go.Bar(x=fs["quarter"],y=fs["ROA"],name='ROA%')
    roe = go.Bar(x=fs["quarter"],y=fs["ROE"],name='ROE%')
    fig = go.Figure(data = [roa,roe],layout=layout)
    # Add a title to the plot
    fig.update_layout(
        title=name+" "+"ROA% ROE% by Year",
        hovermode='x'
    )
    fig.update_xaxes(tickangle=60)

    return fig

def plot_finance(stock_symbol):
    stock_data = Stock(stock_symbol)
    fs = stock_data.financial_statement()
    if len(fs) == 40:
        fs.drop(index=[39],inplace=True)
    pics = {}
    # for topic in ["Total Asset","Net Profit","Dividend Yield","EPS"]:
    #     pics[topic] = finance_scater_plot_1_pic(stock_symbol,fs,topic)
    # pics["ROA & ROE"] = finance_bar_plot_1_pic(stock_symbol,fs,["ROA","ROE"]),0,0,0
    pics = finance_scater_plot_1_pic(stock_symbol,fs,"Net Profit")
    return pics

l = ['GLOBAL', 'CHG', 'CWT', 'EA', 'SUPER', 'JMART', 'RAM', 'BCH', 'CSP', 'APURE', 'SIS', 'AIE', 'APCS', 'LANNA', 'EP', 'AMC', 'IVL', 'QTC', 'METCO', 'NEW']
for i in l:
    a = plot_finance(i)
    a[0].show()
    print("slope: " ,a[1])
    print("intercept: ",a[3])
    print("percent: " ,a[2])

for i in a.keys():
    a[i][0].show()
    print("slope: " ,a[i][1])
    print("intercept: ",a[i][3])
    print("percent: " ,a[i][2])


In [ ]:
import plotly.graph_objects as go

import pandas as pd

# Load data
df = pd.read_csv(
    "https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv")
df.columns = [col.replace("AAPL.", "") for col in df.columns]

# Create figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(df.Date), y=list(df.High)))

# Set title
fig.update_layout(
    title_text="Time series with range slider and selectors"
)


fig.update_xaxes(
    range=[df["Date"].iloc[-90], df["Date"].iloc[-1]]  # set the initial range to the last 3 months
)




fig.show()

In [15]:
from data.function import Categories,Stock
Set = Categories('SET')
Nasdaq = Categories('NASDAQ')
Crypto = Categories('CRYPTO')
all_symbol = []
for symbol in Set.get_all_stock():
    all_symbol.append(symbol+'\t\t- SET')
    print(symbol+'\t\t- SET')
for symbol in Nasdaq.get_all_stock():
    all_symbol.append(symbol+'\t\t- NASDAQ')
    print(symbol+'\t\t- NASDAQ')
for symbol in Crypto.get_all_stock():
    all_symbol.append(symbol+'\t\t- CRYPTO')
    print(symbol+'\t\t- CRYPTO')

2S		- SET
3K-BAT		- SET
7UP		- SET
A		- SET
AAI		- SET
AAV		- SET
ACC		- SET
ACE		- SET
ACG		- SET
ADVANC		- SET
AEONTS		- SET
AFC		- SET
AGE		- SET
AH		- SET
AHC		- SET
AI		- SET
AIE		- SET
AIMCG		- SET
AIMIRT		- SET
AIT		- SET
AJ		- SET
AJA		- SET
AKR		- SET
ALLA		- SET
ALLY		- SET
ALT		- SET
ALUCON		- SET
AMANAH		- SET
AMARIN		- SET
AMATA		- SET
AMATAR		- SET
AMATAV		- SET
AMC		- SET
AMR		- SET
ANAN		- SET
AOT		- SET
AP		- SET
APCO		- SET
APCS		- SET
APEX		- SET
APURE		- SET
AQ		- SET
AQUA		- SET
AS		- SET
ASAP		- SET
ASEFA		- SET
ASIA		- SET
ASIAN		- SET
ASIMAR		- SET
ASK		- SET
ASP		- SET
ASW		- SET
AURA		- SET
AWC		- SET
AYUD		- SET
B		- SET
B-WORK		- SET
B52		- SET
BA		- SET
BAFS		- SET
BAM		- SET
BANPU		- SET
BAREIT		- SET
BAY		- SET
BBGI		- SET
BBL		- SET
BCH		- SET
BCP		- SET
BCPG		- SET
BCT		- SET
BDMS		- SET
BEAUTY		- SET
BEC		- SET
BEM		- SET
BEYOND		- SET
BGC		- SET
BGRIM		- SET
BH		- SET
BIG		- SET
BIOTEC		- SET
BIZ		- SET
BJC		- SET
BJCHI		- SET
BKD		- SET
BKI		- SET
BK